## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [10]:

def build_mlp(input_shape, output_units=10, num_neurons=[64, 32, 16]):
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    
    for idx, n in enumerate(num_neurons):
        
        if idx == 0:
            x = keras.layers.Dense(units=n, activation="relu", name="hidden_layer%d" % (idx))(input_layer)
        else:
            x = keras.layers.Dense(units=n, activation="relu", name="hidden_layer%d" % (idx))(x)
            
    output_layer = keras.layers.Dense(units=output_units, activation="softmax", name="output_layer")(x)
        
    model = keras.models.Model(inputs=input_layer, outputs=output_layer)
    
    return model

In [14]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
BATCH_SIZE=1024
EPOCHS=20
MOMENTUM=0.95

In [15]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
model_ckpt = keras.callbacks.ModelCheckpoint(
    filepath="./Day_086_HW_model.h5", 
    verbose=True,
    monitor="val_loss", 
    save_best_only=True)

results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[model_ckpt]
         )

# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["acc"]
valid_acc = model.history.history["val_acc"]

# Load back
model = keras.models.load_model("./Day_086_HW_model.h5")
loss_loadback, acc_loadback = model.evaluate(x_test, y_test) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer0 (Dense)        (None, 64)                196672    
_________________________________________________________________
hidden_layer1 (Dense)        (None, 32)                2080      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 16)                528       
_________________________________________________________________
output_layer (Dense)         (None, 10)                170       
Total params: 199,450
Trainable params: 199,450
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 4s 79us/step - loss: 2.3020 

In [17]:
# Load back
model = keras.models.load_model("./Day_086_HW_model.h5")
loss_loadback, acc_loadback = model.evaluate(x_test, y_test) 
print(loss_loadback, acc_loadback)

10000/10000 [==============================] - 1s 65us/step
1.7321756381988525 0.3817


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
"""Code Here
將結果繪出
"""